In [ ]:
# Import all library
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D

In [ ]:
# Load datasets
dataset1_path = '/content/drive/MyDrive/Untitled folder/Custom dataset/Face/Preporcessed'
dataset2_path = '/content/drive/MyDrive/Untitled folder/CK+48/CK+48'
# Define face image size
img_width, img_height = 48,48

# batch size
batch_size = 32
# Create ImageDataGenerator instances for both datasets
datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

train_generator1 = datagen.flow_from_directory(
    dataset1_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

train_generator2 = datagen.flow_from_directory(
    dataset2_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

Found 52 images belonging to 7 classes.
Found 981 images belonging to 7 classes.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#create Combined generator
def combined_generator(gen1, gen2):
    while True:
        try:
            x1, y1 = next(gen1)  # Try to get data from generator1
        except StopIteration:
            x1, y1 = None, None  # If generator1 is exhausted, set to None
        try:
            x2, y2 = next(gen2)  # Try to get data from generator2
        except StopIteration:
            break  # If generator2 is exhausted, break the loop

        if x1 is not None and x2 is not None:  # If both generators have data
            yield np.concatenate([x1, x2]), np.concatenate([y1, y2])

combined_generator = combined_generator(train_generator1, train_generator2)



In [ ]:
# Import required library
import numpy as np
from tensorflow.keras.models import Model

In [ ]:
# Load the pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
from tensorflow.keras.layers import Dropout

for layer in base_model.layers[-10:]:    # Freezing the base model layers
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(7, activation='softmax')(x)  #number of layers as per classes for emotions


model1 = Model(inputs=base_model.input, outputs=x)  # Creating the final model


model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])  # Compile the model
for _ in range(len(train_generator1) + len(train_generator2)):
    next(combined_generator)  # Consume data until exhaustion
# fitting the model
model1.fit(combined_generator,
          steps_per_epoch=max(len(train_generator1), len(train_generator2)),
          epochs=5,
          validation_data=combined_generator,
          validation_steps=max(len(train_generator1), len(train_generator2)))

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/5
31/31 ━━━━━━━━━━━━━━━━━━━━ 287s 8s/step - accuracy: 0.4119 - loss: 1.8040 - val_accuracy: 0.1028 - val_loss: 116.0117
Epoch 2/5
31/31 ━━━━━━━━━━━━━━━━━━━━ 250s 8s/step - accuracy: 0.8129 - loss: 0.6135 - val_accuracy: 0.1004 - val_loss: 319.1422
Epoch 3/5
31/31 ━━━━━━━━━━━━━━━━━━━━ 247s 8s/step - accuracy: 0.8746 - loss: 0.4169 - val_accuracy: 0.1093 - val_loss: 3.8572
Epoch 4/5
31/31 ━━━━━━━━━━━━━━━━━━━━ 249s 8s/step - accuracy: 0.9232 - loss: 0.2743 - val_accuracy: 0.1735 - val_loss: 2.7638
Epoch 5/5
31/31 ━━━━━━━━━━━━━━━━━━━━ 248s 8s/step - accuracy: 0.9297 - loss: 0.2791 - val_accuracy: 0.1037 - val_loss: 4.1696


In [ ]:
# Extract the base model's input and output layers
feature_extraction_model = Model(inputs=base_model.input, outputs=base_model.output)

In [ ]:
# Assuming data is preprocessed and batched
features = feature_extraction_model.predict(train_generator2)

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


31/31 ━━━━━━━━━━━━━━━━━━━━ 14s 427ms/step


In [ ]:
#importing library
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
data_dir = '/content/drive/MyDrive/Untitled folder/Custom dataset/Game scne/Test4'    # Define data directory path

# Create ImageDataGenerator for data augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Create training data generator
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(150, 100),
    batch_size=8,
    class_mode='categorical'
)

Found 193 images belonging to 7 classes.


In [ ]:
for data_batch, labels_batch in train_generator:
    print(data_batch.shape)  # verify batch size
    print(labels_batch.shape)  # verify batch size
    break

(8, 150, 100, 3)
(8, 7)


In [ ]:
# Load Pretrained ResNet model without the top layers
base_model2 = ResNet50(weights='imagenet', include_top=False, input_shape=(150, 100, 3))
# Freeze the base model layers
for layer in base_model2.layers:
    layer.trainable = False

In [ ]:
# Add custom top layers
x = base_model2.output
x = Flatten()(x)
x = Dense(126, activation='relu')(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(7, activation='softmax')(x)  #number of layers as per classes for emotions

# Create the final model
model2 = Model(inputs=base_model2.input, outputs=predictions)

In [ ]:
# Compile the model
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'] )
# Fitting the model
history = model2.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs= 5  # Adjust the number of epochs as need
)

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


25/25 ━━━━━━━━━━━━━━━━━━━━ 26s 637ms/step - accuracy: 0.3832 - loss: 3.6516
Epoch 2/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/5


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


25/25 ━━━━━━━━━━━━━━━━━━━━ 15s 593ms/step - accuracy: 0.7922 - loss: 0.8292
Epoch 4/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 539us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 15s 585ms/step - accuracy: 0.8162 - loss: 0.7192


In [ ]:
# Extract the base model's input and output layers
feature_extraction_model2 = Model(inputs=base_model2.input, outputs= base_model2.output)

In [ ]:
# Assuming your data is preprocessed and batched
features2 = feature_extraction_model2.predict(train_generator)

25/25 ━━━━━━━━━━━━━━━━━━━━ 18s 670ms/step


In [ ]:
print(features2.shape)
print(features.shape)

(193, 5, 4, 2048)
(981, 2, 2, 2048)


In [ ]:
# Import required library
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, TimeDistributed, Flatten
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Assume game_features and face_features are your extracted features
game_features = features2  # Example shape
face_features = features  # Example shape

# Pad the facial features to match the game features sequence length
face_features_padded = pad_sequences(face_features, maxlen=5, dtype='float32', padding='post', truncating='post')

In [ ]:
#Reshape the features to have the same sequence length
game_features_reshaped = game_features.reshape(game_features.shape[0], game_features.shape[1], -1)  # (193, 5, 4*2048)
face_features_reshaped = face_features_padded.reshape(face_features_padded.shape[0], face_features_padded.shape[1], -1)  # (981, 5, 2*2048)

# Align the number of samples
min_samples = min(game_features_reshaped.shape[0], face_features_reshaped.shape[0])

# Truncate both feature sets to the minimum sample size
game_features_aligned = game_features_reshaped[:min_samples]
face_features_aligned = face_features_reshaped[:min_samples]

In [ ]:
#combine learnt features
combined_features = np.concatenate((game_features_aligned, face_features_aligned), axis=-1)


In [ ]:
num_samples = combined_features.shape[0]  # Number of samples after truncation
num_classes = 7
labels = np.zeros(num_samples)
labels[num_samples // 2:] = 1


In [ ]:
model4 = Sequential()
model4.add(LSTM(256, return_sequences=True, input_shape=(combined_features.shape[1], combined_features.shape[2])))
model4.add(Dropout(0.5))
model4.add(LSTM(128))
model4.add(Dropout(0.5))
model4.add(Dense(1, activation='sigmoid'))  # Adjust output layer based on task for future adjust according to future dataset classes

# Compile the model
model4.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model4.fit(combined_features, labels, epochs=5, batch_size=32)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 746ms/step - accuracy: 0.5322 - loss: 0.7587
Epoch 2/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 585ms/step - accuracy: 0.6533 - loss: 0.6341
Epoch 3/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 709ms/step - accuracy: 0.6558 - loss: 0.5980
Epoch 4/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 659ms/step - accuracy: 0.7630 - loss: 0.5264
Epoch 5/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 723ms/step - accuracy: 0.8066 - loss: 0.4721


In [ ]:
model4.summary()

Model: "sequential_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_26 (LSTM)                       │ (None, 5, 256)              │      12,846,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_21 (Dropout)                 │ (None, 5, 256)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_27 (LSTM)                       │ (None, 128)                 │         197,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_22 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_27 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 39,129,989 (149.27 MB)

 Trainable params: 13,043,329 (49.76 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 26,086,660 (99.51 MB)